In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import pickle
import warnings
from datetime import datetime, timedelta
import cupy as cp
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
warnings.filterwarnings('ignore')

# Data processing and ML
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

# Sentiment Analysis
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class LSTMModel(nn.Module):
    """PyTorch LSTM Model for stock prediction"""
    def __init__(self, input_size, lstm_units=50, num_layers=3, dropout_rate=0.2):
        super(LSTMModel, self).__init__()
        
        self.num_layers = num_layers
        self.lstm_units = lstm_units
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=lstm_units,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Dense layers
        self.fc1 = nn.Linear(lstm_units, 25)
        self.fc2 = nn.Linear(25, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Initialize hidden and cell states
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.lstm_units).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.lstm_units).to(x.device)
        
        # LSTM forward pass
        lstm_out, _ = self.lstm(x, (h0, c0))
        
        # Take the output from the last time step
        lstm_out = lstm_out[:, -1, :]
        
        # Apply dropout
        lstm_out = self.dropout(lstm_out)
        
        # Dense layers
        out = self.relu(self.fc1(lstm_out))
        out = self.dropout(out)
        out = self.fc2(out)
        
        return out

class StockForecasterPyTorch:
    def __init__(self, symbol):
        self.symbol = symbol
        self.data = None
        self.arima_model = None
        self.lstm_model = None
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.arima_residuals = None
        self.predictions = None
        self.news_api_key = "02cc51bf98a740e19221e22ef87f6d56"  # Replace with your News API key
        self.device = device
        
        # CuPy arrays for GPU acceleration
        self.use_cupy = cp.cuda.is_available()
        if self.use_cupy:
            print("CuPy GPU acceleration enabled")
        else:
            print("CuPy not available, using CPU")

    def cupy_to_numpy(self, arr):
        """Convert CuPy array to NumPy if needed"""
        if self.use_cupy and isinstance(arr, cp.ndarray):
            return cp.asnumpy(arr)
        return arr

    def numpy_to_cupy(self, arr):
        """Convert NumPy array to CuPy if possible"""
        if self.use_cupy and isinstance(arr, np.ndarray):
            return cp.asarray(arr)
        return arr

    def fetch_stock_data(self, period="2y"):
        """Fetch stock data from Yahoo Finance"""
        try:
            ticker = yf.Ticker(self.symbol)
            self.data = ticker.history(period=period)

            # Add technical indicators using CuPy for acceleration
            close_prices = self.numpy_to_cupy(self.data['Close'].values)
            
            # Moving averages
            ma_20 = self.cupy_to_numpy(self.calculate_ma_cupy(close_prices, 20))
            ma_50 = self.cupy_to_numpy(self.calculate_ma_cupy(close_prices, 50))
            
            self.data['MA_20'] = ma_20
            self.data['MA_50'] = ma_50
            self.data['RSI'] = self.calculate_rsi(self.data['Close'])
            
            # Volatility using CuPy
            returns = cp.diff(close_prices) / close_prices[:-1] if self.use_cupy else np.diff(close_prices) / close_prices[:-1]
            volatility = self.cupy_to_numpy(self.rolling_std_cupy(returns, 20)) if self.use_cupy else pd.Series(returns).rolling(20).std().values
            self.data['Volatility'] = np.concatenate([[np.nan], volatility])

            print(f"Fetched {len(self.data)} days of data for {self.symbol}")
            return True
        except Exception as e:
            print(f"Error fetching data: {e}")
            return False

    def calculate_ma_cupy(self, prices, window):
        """Calculate moving average using CuPy"""
        if not self.use_cupy:
            return pd.Series(prices).rolling(window).mean().values
        
        ma = cp.full_like(prices, cp.nan)
        for i in range(window - 1, len(prices)):
            ma[i] = cp.mean(prices[i - window + 1:i + 1])
        return ma

    def rolling_std_cupy(self, arr, window):
        """Calculate rolling standard deviation using CuPy"""
        if not self.use_cupy:
            return pd.Series(arr).rolling(window).std().values
        
        std = cp.full(len(arr) - window + 1, cp.nan)
        for i in range(window - 1, len(arr)):
            std[i - window + 1] = cp.std(arr[i - window + 1:i + 1])
        return std

    def calculate_rsi(self, prices, window=14):
        """Calculate RSI indicator"""
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def fetch_news_sentiment(self, days_back=7):
        """Fetch news and calculate sentiment score"""
        try:
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days_back)

            url = f"https://newsapi.org/v2/everything"
            params = {
                'q': f"{self.symbol} OR stock market OR economy",
                'from': start_date.strftime('%Y-%m-%d'),
                'to': end_date.strftime('%Y-%m-%d'),
                'sortBy': 'relevancy',
                'apiKey': self.news_api_key,
                'language': 'en',
                'pageSize': 50
            }

            response = requests.get(url, params=params)
            news_data = response.json()

            if news_data['status'] == 'ok':
                articles = news_data['articles']
                sentiments = []
                analyzer = SentimentIntensityAnalyzer()

                for article in articles:
                    title = article.get('title', '')
                    description = article.get('description', '')
                    text = f"{title} {description}"

                    # VADER sentiment
                    vader_score = analyzer.polarity_scores(text)['compound']

                    # TextBlob sentiment
                    blob = TextBlob(text)
                    textblob_score = blob.sentiment.polarity

                    # Average sentiment
                    avg_sentiment = (vader_score + textblob_score) / 2
                    sentiments.append(avg_sentiment)

                overall_sentiment = np.mean(sentiments) if sentiments else 0
                print(f"News sentiment score: {overall_sentiment:.3f} (from {len(articles)} articles)")
                return overall_sentiment, articles[:10]  # Return top 10 articles
            else:
                print("No news data available")
                return 0, []
        except Exception as e:
            print(f"Error fetching news sentiment: {e}")
            return 0, []

    def optimize_arima_parameters(self, max_p=5, max_d=2, max_q=5):
        """Optimize ARIMA parameters using grid search and information criteria"""
        print("Optimizing ARIMA hyperparameters...")

        # Check stationarity first
        from statsmodels.tsa.stattools import adfuller

        ts_data = self.data['Close'].dropna()
        adf_result = adfuller(ts_data)
        print(f"ADF Statistic: {adf_result[0]:.6f}")
        print(f"p-value: {adf_result[1]:.6f}")

        is_stationary = adf_result[1] <= 0.05
        print(f"Series is {'stationary' if is_stationary else 'non-stationary'}")

        # Grid search for optimal parameters
        best_aic = float('inf')
        best_bic = float('inf')
        best_order_aic = None
        best_order_bic = None
        results = []

        print("\nGrid search for optimal ARIMA parameters:")
        print("p\td\tq\tAIC\t\tBIC\t\tHQIC\t\tLLF")
        print("-" * 70)

        for p in range(0, max_p + 1):
            for d in range(0, max_d + 1):
                for q in range(0, max_q + 1):
                    try:
                        # Skip if d is too high for stationary series
                        if is_stationary and d > 1:
                            continue

                        # Fit ARIMA model
                        model = ARIMA(ts_data, order=(p, d, q))
                        fitted_model = model.fit()

                        aic = fitted_model.aic
                        bic = fitted_model.bic
                        hqic = fitted_model.hqic
                        llf = fitted_model.llf

                        results.append({
                            'order': (p, d, q),
                            'aic': aic,
                            'bic': bic,
                            'hqic': hqic,
                            'llf': llf,
                            'model': fitted_model
                        })

                        print(f"{p}\t{d}\t{q}\t{aic:.4f}\t\t{bic:.4f}\t\t{hqic:.4f}\t\t{llf:.4f}")

                        # Track best models
                        if aic < best_aic:
                            best_aic = aic
                            best_order_aic = (p, d, q)

                        if bic < best_bic:
                            best_bic = bic
                            best_order_bic = (p, d, q)

                    except Exception as e:
                        continue

        # Select best model based on AIC (generally preferred for forecasting)
        best_result = min(results, key=lambda x: x['aic'])
        best_order = best_result['order']

        print(f"\n🎯 Optimization Results:")
        print(f"Best order by AIC: {best_order_aic} (AIC: {best_aic:.4f})")
        print(f"Best order by BIC: {best_order_bic} (BIC: {best_bic:.4f})")
        print(f"Selected order: {best_order}")

        return best_result['model'], best_order, results

    def fit_arima(self):
        """Fit ARIMA model to capture linear trends with hyperparameter optimization"""
        try:
            print("Fitting ARIMA model with hyperparameter optimization...")

            # First try automated approach
            try:
                print("\n1️⃣ Trying auto_arima for quick optimization...")
                auto_model = auto_arima(
                    self.data['Close'].dropna(),
                    start_p=0, start_q=0, start_P=0, start_Q=0,
                    test='adf',
                    max_p=5, max_q=5, max_P=2, max_Q=2,
                    seasonal=True, m=12,  # Monthly seasonality
                    stepwise=True,
                    suppress_warnings=True,
                    error_action='ignore',
                    trace=True,
                    information_criterion='aic',
                    out_of_sample_size=int(len(self.data) * 0.2)  # 20% for validation
                )

                auto_order = auto_model.order
                auto_seasonal_order = auto_model.seasonal_order if hasattr(auto_model, 'seasonal_order') else (0, 0, 0, 0)
                auto_aic = auto_model.aic()

                print(f"Auto-ARIMA order: {auto_order}")
                print(f"Auto-ARIMA seasonal order: {auto_seasonal_order}")
                print(f"Auto-ARIMA AIC: {auto_aic:.4f}")

            except Exception as e:
                print(f"Auto-ARIMA failed: {e}")
                auto_model = None
                auto_aic = float('inf')

            # Manual grid search optimization
            print("\n2️⃣ Manual grid search optimization...")
            manual_model, manual_order, all_results = self.optimize_arima_parameters()
            manual_aic = manual_model.aic

            # Compare and select best model
            if auto_model is not None and auto_aic < manual_aic:
                print(f"\n✅ Selected: Auto-ARIMA (AIC: {auto_aic:.4f})")
                self.arima_model = ARIMA(
                    self.data['Close'].dropna(),
                    order=auto_order,
                    seasonal_order=auto_seasonal_order
                ).fit()
                final_order = auto_order
            else:
                print(f"\n✅ Selected: Manual optimization (AIC: {manual_aic:.4f})")
                self.arima_model = manual_model
                final_order = manual_order

            # Validate model diagnostics
            self.validate_arima_model()

            # Get residuals for LSTM
            fitted_values = self.arima_model.fittedvalues
            self.arima_residuals = self.data['Close'].dropna() - fitted_values

            # Calculate residual statistics
            residual_mean = self.arima_residuals.mean()
            residual_std = self.arima_residuals.std()

            print(f"\n📊 Final ARIMA Model Summary:")
            print(f"Order: {final_order}")
            print(f"AIC: {self.arima_model.aic:.4f}")
            print(f"BIC: {self.arima_model.bic:.4f}")
            print(f"Log-likelihood: {self.arima_model.llf:.4f}")
            print(f"Residual mean: {residual_mean:.6f}")
            print(f"Residual std: {residual_std:.4f}")

            return True

        except Exception as e:
            print(f"Error fitting ARIMA: {e}")
            return False

    def validate_arima_model(self):
        """Validate ARIMA model with diagnostic tests"""
        try:
            from statsmodels.stats.diagnostic import acorr_ljungbox
            from statsmodels.stats.stattools import jarque_bera

            residuals = self.arima_model.resid

            # Ljung-Box test for autocorrelation in residuals
            lb_stat, lb_pvalue = acorr_ljungbox(residuals, lags=10, return_df=False)

            # Jarque-Bera test for normality
            jb_stat, jb_pvalue, _, _ = jarque_bera(residuals)
            print(f"\n🔍 ARIMA Model Diagnostics:")
            lb_pval = float(lb_pvalue[-1]) if lb_pvalue[-1] is not None else float('nan')
            jb_pval = float(jb_pvalue) if jb_pvalue is not None else float('nan')

            print(f"Ljung-Box test (autocorrelation): p-value = {lb_pval:.4f}")
            print(f"{'✅ No autocorrelation' if lb_pval > 0.05 else '⚠️ Autocorrelation detected'}")

            print(f"Jarque-Bera test (normality): p-value = {jb_pval:.4f}")
            print(f"{'✅ Residuals are normal' if jb_pval > 0.05 else '⚠️ Residuals are not normal'}")

            return {
                'ljung_box_pvalue': lb_pvalue[-1],
                'jarque_bera_pvalue': jb_pvalue,
                'autocorrelation_ok': lb_pvalue[-1] > 0.05,
                'normality_ok': jb_pvalue > 0.05
            }

        except Exception as e:
            print(f"Error in model validation: {e}")
            return None

    def prepare_lstm_data(self, look_back=60):
        """Prepare data for LSTM training using CuPy acceleration"""
        try:
            # Combine residuals with technical indicators
            features = pd.DataFrame({
                'residuals': self.arima_residuals,
                'volume': self.data['Volume'][self.arima_residuals.index],
                'rsi': self.data['RSI'][self.arima_residuals.index],
                'volatility': self.data['Volatility'][self.arima_residuals.index]
            }).fillna(method='ffill').dropna()

            # Scale features
            scaled_features = self.scaler.fit_transform(features)

            # Use CuPy for sequence creation if available
            if self.use_cupy:
                scaled_features_gpu = cp.asarray(scaled_features)
                X_gpu = cp.zeros((len(scaled_features) - look_back, look_back, scaled_features.shape[1]))
                y_gpu = cp.zeros(len(scaled_features) - look_back)
                
                for i in range(look_back, len(scaled_features)):
                    X_gpu[i - look_back] = scaled_features_gpu[i - look_back:i]
                    y_gpu[i - look_back] = scaled_features_gpu[i, 0]  # Predict residuals
                
                X = cp.asnumpy(X_gpu)
                y = cp.asnumpy(y_gpu)
            else:
                # Create sequences using NumPy
                X, y = [], []
                for i in range(look_back, len(scaled_features)):
                    X.append(scaled_features[i-look_back:i])
                    y.append(scaled_features[i, 0])  # Predict residuals
                X, y = np.array(X), np.array(y)

            return X, y
        except Exception as e:
            print(f"Error preparing LSTM data: {e}")
            return None, None

    def optimize_lstm_hyperparameters(self, X_train, y_train, X_val, y_val):
        """Optimize LSTM hyperparameters using PyTorch"""
        print("Optimizing LSTM hyperparameters...")

        # Hyperparameter grid
        param_grid = {
            'lstm_units': [32, 50, 64, 100],
            'dropout_rate': [0.1, 0.2, 0.3],
            'learning_rate': [0.001, 0.01, 0.1],
            'batch_size': [16, 32, 64],
            'num_layers': [2, 3, 4]
        }

        best_score = float('inf')
        best_params = None
        results = []

        # Convert to PyTorch tensors
        X_train_tensor = torch.FloatTensor(X_train).to(self.device)
        y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(self.device)
        X_val_tensor = torch.FloatTensor(X_val).to(self.device)
        y_val_tensor = torch.FloatTensor(y_val.reshape(-1, 1)).to(self.device)

        # Random search (to avoid exhaustive grid search)
        import random
        n_trials = 20  # Number of random combinations to try

        print("Trying random hyperparameter combinations:")
        print("Trial\tUnits\tLayers\tDropout\tLR\tBatch\tVal Loss")
        print("-" * 60)

        for trial in range(n_trials):
            params = {
                'lstm_units': random.choice(param_grid['lstm_units']),
                'dropout_rate': random.choice(param_grid['dropout_rate']),
                'learning_rate': random.choice(param_grid['learning_rate']),
                'batch_size': random.choice(param_grid['batch_size']),
                'num_layers': random.choice(param_grid['num_layers'])
            }

            try:
                # Build model with current parameters
                model = LSTMModel(
                    input_size=X_train.shape[2],
                    lstm_units=params['lstm_units'],
                    num_layers=params['num_layers'],
                    dropout_rate=params['dropout_rate']
                ).to(self.device)

                # Define loss and optimizer
                criterion = nn.MSELoss()
                optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

                # Create data loaders
                train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
                train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)

                # Train model
                model.train()
                for epoch in range(50):  # Reduced epochs for hyperparameter search
                    epoch_loss = 0
                    for batch_X, batch_y in train_loader:
                        optimizer.zero_grad()
                        outputs = model(batch_X)
                        loss = criterion(outputs, batch_y)
                        loss.backward()
                        optimizer.step()
                        epoch_loss += loss.item()

                # Evaluate on validation set
                model.eval()
                with torch.no_grad():
                    val_outputs = model(X_val_tensor)
                    val_loss = criterion(val_outputs, y_val_tensor).item()

                results.append({
                    'params': params.copy(),
                    'val_loss': val_loss,
                    'model': model
                })

                print(f"{trial+1:2d}\t{params['lstm_units']:3d}\t{params['num_layers']}\t{params['dropout_rate']:.1f}\t{params['learning_rate']:.3f}\t{params['batch_size']:2d}\t{val_loss:.6f}")

                if val_loss < best_score:
                    best_score = val_loss
                    best_params = params.copy()

            except Exception as e:
                print(f"{trial+1:2d}\tFAILED: {str(e)[:30]}")
                continue

        print(f"\n🎯 Best LSTM Parameters:")
        for key, value in best_params.items():
            print(f"{key}: {value}")
        print(f"Best validation loss: {best_score:.6f}")

        return best_params, results

    def train_lstm(self, epochs=100, batch_size=32):
        """Train LSTM model on residuals with PyTorch and hyperparameter optimization"""
        try:
            print("Preparing LSTM data...")
            X, y = self.prepare_lstm_data()

            if X is None or y is None:
                return False

            # Split data
            split_idx = int(len(X) * 0.7)  # 70% train
            val_split_idx = int(len(X) * 0.85)  # 15% validation, 15% test

            X_train, y_train = X[:split_idx], y[:split_idx]
            X_val, y_val = X[split_idx:val_split_idx], y[split_idx:val_split_idx]
            X_test, y_test = X[val_split_idx:], y[val_split_idx:]

            print(f"Training samples: {len(X_train)}")
            print(f"Validation samples: {len(X_val)}")
            print(f"Test samples: {len(X_test)}")

            # Optimize hyperparameters
            best_params, optimization_results = self.optimize_lstm_hyperparameters(
                X_train, y_train, X_val, y_val
            )

            # Train final model with best parameters
            print(f"\n🔄 Training final LSTM model with best parameters...")
            self.lstm_model = LSTMModel(
                input_size=X.shape[2],
                lstm_units=best_params['lstm_units'],
                num_layers=best_params['num_layers'],
                dropout_rate=best_params['dropout_rate']
            ).to(self.device)

            # Convert to PyTorch tensors
            X_train_tensor = torch.FloatTensor(X_train).to(self.device)
            y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(self.device)
            X_val_tensor = torch.FloatTensor(X_val).to(self.device)
            y_val_tensor = torch.FloatTensor(y_val.reshape(-1, 1)).to(self.device)
            X_test_tensor = torch.FloatTensor(X_test).to(self.device)
            y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1)).to(self.device)

            # Define loss and optimizer
            criterion = nn.MSELoss()
            optimizer = optim.Adam(self.lstm_model.parameters(), lr=best_params['learning_rate'])
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=8, min_lr=1e-7)

            # Create data loaders
            train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
            train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

            # Training loop with early stopping
            best_val_loss = float('inf')
            patience_counter = 0
            patience = 15
            training_history = {'train_loss': [], 'val_loss': []}

            for epoch in range(epochs):
                # Training phase
                self.lstm_model.train()
                train_loss = 0
                for batch_X, batch_y in train_loader:
                    optimizer.zero_grad()
                    outputs = self.lstm_model(batch_X)
                    loss = criterion(outputs, batch_y)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()

                train_loss /= len(train_loader)

                # Validation phase
                self.lstm_model.eval()
                with torch.no_grad():
                    val_outputs = self.lstm_model(X_val_tensor)
                    val_loss = criterion(val_outputs, y_val_tensor).item()

                # Learning rate scheduling
                scheduler.step(val_loss)

                training_history['train_loss'].append(train_loss)
                training_history['val_loss'].append(val_loss)

                # Early stopping
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    patience_counter = 0
                    # Save best model
                    torch.save(self.lstm_model.state_dict(), 'best_lstm_model.pth')
                else:
                    patience_counter += 1

                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

                if (epoch + 1) % 10 == 0:
                    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')

            # Load best model
            self.lstm_model.load_state_dict(torch.load('best_lstm_model.pth'))

            # Final evaluation
            self.lstm_model.eval()
            with torch.no_grad():
                train_outputs = self.lstm_model(X_train_tensor)
                train_loss = criterion(train_outputs, y_train_tensor).item()
                train_mae = torch.mean(torch.abs(train_outputs - y_train_tensor)).item()

                val_outputs = self.lstm_model(X_val_tensor)
                val_loss = criterion(val_outputs, y_val_tensor).item()
                val_mae = torch.mean(torch.abs(val_outputs - y_val_tensor)).item()

                test_outputs = self.lstm_model(X_test_tensor)
                test_loss = criterion(test_outputs, y_test_tensor).item()
                test_mae = torch.mean(torch.abs(test_outputs - y_test_tensor)).item()

            print(f"\n📊 Final LSTM Model Performance:")
            print(f"Training Loss: {train_loss:.6f} | MAE: {train_mae:.6f}")
            print(f"Validation Loss: {val_loss:.6f} | MAE: {val_mae:.6f}")
            print(f"Test Loss: {test_loss:.6f} | MAE: {test_mae:.6f}")

            # Store optimization results
            self.lstm_optimization_results = {
                'best_params': best_params,
                'all_results': optimization_results,
                'final_scores': {
                    'train': (train_loss, train_mae),
                    'val': (val_loss, val_mae),
                    'test': (test_loss, test_mae)
                },
                'training_history': training_history
            }

            return True

        except Exception as e:
            print(f"Error training LSTM: {e}")
            return False

    def forecast(self, days_ahead=30):
        """Generate hybrid forecast using PyTorch model"""
        try:
            print(f"Generating {days_ahead} day forecast...")

            # ARIMA forecast
            arima_forecast = self.arima_model.forecast(steps=days_ahead)
            arima_conf_int = self.arima_model.get_forecast(steps=days_ahead).conf_int()

            # Prepare data for LSTM residual prediction
            last_sequence = self.prepare_last_sequence()

            # LSTM residual forecast using PyTorch
            lstm_residual_forecast = []
            current_sequence = last_sequence.copy()

            # Convert to tensor and move to device
            self.lstm_model.eval()
            with torch.no_grad():
                for _ in range(days_ahead):
                    # Convert to tensor
                    sequence_tensor = torch.FloatTensor(current_sequence).unsqueeze(0).to(self.device)
                    
                    # Predict next residual
                    next_residual = self.lstm_model(sequence_tensor).cpu().numpy()[0, 0]
                    lstm_residual_forecast.append(next_residual)

                    # Update sequence
                    if self.use_cupy:
                        current_sequence_gpu = cp.asarray(current_sequence)
                        current_sequence_gpu = cp.roll(current_sequence_gpu, -1, axis=0)
                        current_sequence_gpu[-1, 0] = next_residual
                        
                        # Update with scaled exogenous features
                        next_date = self.data.index[-1] + timedelta(days=_ + 1)
                        scaled_exog = self.get_scaled_exog_features(next_date)
                        if scaled_exog is not None:
                            current_sequence_gpu[-1, 1:] = cp.asarray(scaled_exog)
                        
                        current_sequence = cp.asnumpy(current_sequence_gpu)
                    else:
                        current_sequence = np.roll(current_sequence, -1, axis=0)
                        current_sequence[-1, 0] = next_residual
                        
                        # Update with scaled exogenous features
                        next_date = self.data.index[-1] + timedelta(days=_ + 1)
                        scaled_exog = self.get_scaled_exog_features(next_date)
                        if scaled_exog is not None:
                            current_sequence[-1, 1:] = scaled_exog

            # Inverse transform residuals
            dummy_features = np.zeros((len(lstm_residual_forecast), self.scaler.scale_.shape[0]))
            dummy_features[:, 0] = lstm_residual_forecast
            lstm_residual_forecast = self.scaler.inverse_transform(dummy_features)[:, 0]

            # Combine ARIMA + LSTM predictions
            hybrid_forecast = arima_forecast + lstm_residual_forecast

            # Get news sentiment and adjust predictions
            sentiment_score, news_articles = self.fetch_news_sentiment()
            sentiment_adjustment = sentiment_score * 0.05  # 5% max adjustment based on sentiment
            adjusted_forecast = hybrid_forecast * (1 + sentiment_adjustment)

            # Create forecast dates
            last_date = self.data.index[-1]
            forecast_dates = pd.date_range(
                start=last_date + timedelta(days=1),
                periods=days_ahead,
                freq='D'
            )

            # Store predictions
            self.predictions = pd.DataFrame({
                'date': forecast_dates,
                'arima_forecast': arima_forecast,
                'hybrid_forecast': hybrid_forecast,
                'adjusted_forecast': adjusted_forecast,
                'lower_bound': arima_conf_int.iloc[:, 0],
                'upper_bound': arima_conf_int.iloc[:, 1],
                'sentiment_score': sentiment_score
            })

            return self.predictions, news_articles, sentiment_score
        except Exception as e:
            print(f"Error generating forecast: {e}")
            return None, [], 0

    def prepare_last_sequence(self, look_back=60):
        """Prepare the last sequence for LSTM prediction using CuPy acceleration"""
        try:
            features = pd.DataFrame({
                'residuals': self.arima_residuals,
                'volume': self.data['Volume'][self.arima_residuals.index],
                'rsi': self.data['RSI'][self.arima_residuals.index],
                'volatility': self.data['Volatility'][self.arima_residuals.index]
            }).fillna(method='ffill').dropna()

            scaled_features = self.scaler.transform(features)
            
            if self.use_cupy:
                # Use CuPy for faster array operations
                scaled_features_gpu = cp.asarray(scaled_features)
                last_sequence = cp.asnumpy(scaled_features_gpu[-look_back:])
            else:
                last_sequence = scaled_features[-look_back:]
                
            return last_sequence
        except Exception as e:
            print(f"Error preparing last sequence: {e}")
            return np.zeros((60, 4))

    def calculate_trading_signals(self, current_price):
        """Calculate buy/sell signals and profit projections with CuPy acceleration"""
        try:
            if self.predictions is None:
                return None

            forecast_prices = self.predictions['adjusted_forecast'].values
            dates = self.predictions['date'].values

            # Use CuPy for faster calculations if available
            if self.use_cupy:
                forecast_gpu = cp.asarray(forecast_prices)
                max_price = float(cp.max(forecast_gpu))
                min_price = float(cp.min(forecast_gpu))
                max_idx = int(cp.argmax(forecast_gpu))
                min_idx = int(cp.argmin(forecast_gpu))
            else:
                max_price = forecast_prices.max()
                min_price = forecast_prices.min()
                max_idx = np.argmax(forecast_prices)
                min_idx = np.argmin(forecast_prices)

            # Trading recommendations
            if current_price < forecast_prices[0]:
                recommendation = "BUY"
                target_price = max_price
                potential_profit = ((target_price - current_price) / current_price) * 100
                optimal_sell_date = dates[max_idx]
            elif current_price > forecast_prices[-1]:
                recommendation = "SELL"
                target_price = min_price
                potential_profit = ((current_price - target_price) / current_price) * 100
                optimal_sell_date = dates[min_idx]
            else:
                recommendation = "HOLD"
                target_price = forecast_prices[-1]
                potential_profit = ((target_price - current_price) / current_price) * 100
                optimal_sell_date = dates[-1]

            return {
                'recommendation': recommendation,
                'current_price': current_price,
                'target_price': target_price,
                'potential_profit_pct': potential_profit,
                'optimal_sell_date': optimal_sell_date,
                'forecast_prices': forecast_prices,
                'dates': dates
            }
        except Exception as e:
            print(f"Error calculating trading signals: {e}")
            return None

    def evaluate_model_performance(self):
        """Comprehensive model evaluation and comparison with CuPy acceleration"""
        try:
            print("\n" + "="*60)
            print("📊 COMPREHENSIVE MODEL EVALUATION")
            print("="*60)

            # Get recent data for backtesting
            test_data = self.data['Close'].tail(60).values  # Last 60 days for testing
            train_data = self.data['Close'][:-60].values

            # 1. Baseline Models
            naive_forecast = [train_data[-1]] * len(test_data)  # Naive: last value
            sma_forecast = [np.mean(train_data[-20:])] * len(test_data)  # Simple moving average

            # 2. ARIMA-only predictions
            arima_only_model = ARIMA(train_data, order=self.arima_model.model.order).fit()
            arima_forecast = arima_only_model.forecast(steps=len(test_data))

            # 3. Hybrid model predictions (ARIMA + LSTM)
            hybrid_predictions = self.generate_backtest_predictions(len(test_data))

            # Calculate metrics using CuPy if available
            def calculate_metrics(actual, predicted):
                if self.use_cupy:
                    actual_gpu = cp.asarray(actual)
                    predicted_gpu = cp.asarray(predicted)
                    
                    mse = float(cp.mean((actual_gpu - predicted_gpu) ** 2))
                    rmse = float(cp.sqrt(mse))
                    mae = float(cp.mean(cp.abs(actual_gpu - predicted_gpu)))
                    mape = float(cp.mean(cp.abs((actual_gpu - predicted_gpu) / actual_gpu)) * 100)

                    # Directional accuracy
                    actual_direction = cp.sign(cp.diff(actual_gpu))
                    pred_direction = cp.sign(cp.diff(predicted_gpu))
                    directional_accuracy = float(cp.mean(actual_direction == pred_direction) * 100)
                else:
                    mse = np.mean((actual - predicted) ** 2)
                    rmse = np.sqrt(mse)
                    mae = np.mean(np.abs(actual - predicted))
                    mape = np.mean(np.abs((actual - predicted) / actual)) * 100

                    # Directional accuracy
                    actual_direction = np.sign(np.diff(actual))
                    pred_direction = np.sign(np.diff(predicted))
                    directional_accuracy = np.mean(actual_direction == pred_direction) * 100

                return {
                    'MSE': mse,
                    'RMSE': rmse,
                    'MAE': mae,
                    'MAPE': mape,
                    'Directional_Accuracy': directional_accuracy
                }

            # Evaluate all models
            models_performance = {
                'Naive': calculate_metrics(test_data, naive_forecast),
                'Moving_Average': calculate_metrics(test_data, sma_forecast),
                'ARIMA_Only': calculate_metrics(test_data, arima_forecast),
                'Hybrid_ARIMA_LSTM': calculate_metrics(test_data, hybrid_predictions)
            }

            # Display results
            print("\n📈 Model Comparison Results:")
            print("-" * 80)
            print(f"{'Model':<20} {'RMSE':<10} {'MAE':<10} {'MAPE':<10} {'Dir_Acc':<10}")
            print("-" * 80)

            best_model = None
            best_rmse = float('inf')

            for model_name, metrics in models_performance.items():
                print(f"{model_name:<20} {metrics['RMSE']:<10.4f} {metrics['MAE']:<10.4f} "
                      f"{metrics['MAPE']:<10.2f}% {metrics['Directional_Accuracy']:<10.1f}%")

                if metrics['RMSE'] < best_rmse:
                    best_rmse = metrics['RMSE']
                    best_model = model_name

            print("-" * 80)
            print(f"🏆 Best performing model: {best_model}")

            # Statistical significance test
            self.statistical_significance_test(test_data, arima_forecast, hybrid_predictions)

            # Store evaluation results
            self.model_evaluation = {
                'performance_metrics': models_performance,
                'best_model': best_model,
                'test_period_length': len(test_data)
            }

            return models_performance

        except Exception as e:
            print(f"Error in model evaluation: {e}")
            return None

    def statistical_significance_test(self, actual, arima_pred, hybrid_pred):
        """Perform statistical significance test between models with CuPy acceleration"""
        try:
            from scipy.stats import ttest_rel

            if self.use_cupy:
                actual_gpu = cp.asarray(actual)
                arima_pred_gpu = cp.asarray(arima_pred)
                hybrid_pred_gpu = cp.asarray(hybrid_pred)
                
                arima_errors = cp.asnumpy(cp.abs(actual_gpu - arima_pred_gpu))
                hybrid_errors = cp.asnumpy(cp.abs(actual_gpu - hybrid_pred_gpu))
            else:
                arima_errors = np.abs(actual - arima_pred)
                hybrid_errors = np.abs(actual - hybrid_pred)

            # Paired t-test
            t_stat, p_value = ttest_rel(arima_errors, hybrid_errors)

            print(f"\n🔬 Statistical Significance Test (Paired t-test):")
            print(f"H0: No difference between ARIMA and Hybrid model errors")
            print(f"t-statistic: {t_stat:.4f}")
            print(f"p-value: {p_value:.4f}")

            if p_value < 0.05:
                winner = "Hybrid" if np.mean(hybrid_errors) < np.mean(arima_errors) else "ARIMA"
                print(f"✅ Significant difference detected (p < 0.05). {winner} model is significantly better.")
            else:
                print("❌ No significant difference between models (p >= 0.05)")

        except Exception as e:
            print(f"Error in significance test: {e}")

    def generate_backtest_predictions(self, n_steps):
        """Generate predictions for backtesting using PyTorch model"""
        try:
            # Get ARIMA predictions
            arima_pred = self.arima_model.forecast(steps=n_steps)

            # Simulate LSTM residual predictions using the trained PyTorch model
            recent_residuals = self.arima_residuals.tail(20).values
            
            if self.use_cupy:
                recent_residuals_gpu = cp.asarray(recent_residuals)
                residual_pattern = float(cp.mean(recent_residuals_gpu))
            else:
                residual_pattern = np.mean(recent_residuals)

            # Simple residual forecast (this would be replaced by actual LSTM predictions)
            lstm_residuals = [residual_pattern * (0.9 ** i) for i in range(n_steps)]

            # Combine predictions
            if self.use_cupy:
                arima_pred_gpu = cp.asarray(arima_pred)
                lstm_residuals_gpu = cp.asarray(lstm_residuals)
                hybrid_pred = cp.asnumpy(arima_pred_gpu + lstm_residuals_gpu)
            else:
                hybrid_pred = arima_pred + lstm_residuals

            return hybrid_pred

        except Exception as e:
            print(f"Error generating backtest predictions: {e}")
            return np.zeros(n_steps)

    def save_model(self, filename):
        """Save the trained PyTorch model and associated data"""
        try:
            model_data = {
                'symbol': self.symbol,
                'arima_model': self.arima_model,
                'lstm_model_state_dict': self.lstm_model.state_dict() if self.lstm_model else None,
                'lstm_model_params': {
                    'input_size': 4,  # Adjust based on your features
                    'lstm_units': getattr(self, 'lstm_optimization_results', {}).get('best_params', {}).get('lstm_units', 50),
                    'num_layers': getattr(self, 'lstm_optimization_results', {}).get('best_params', {}).get('num_layers', 3),
                    'dropout_rate': getattr(self, 'lstm_optimization_results', {}).get('best_params', {}).get('dropout_rate', 0.2)
                },
                'scaler': self.scaler,
                'last_date': self.data.index[-1],
                'last_price': self.data['Close'].iloc[-1],
                'predictions': self.predictions,
                'data_tail': self.data.tail(100),  # Keep last 100 days for reference
                'arima_optimization_results': getattr(self, 'arima_optimization_results', None),
                'lstm_optimization_results': getattr(self, 'lstm_optimization_results', None),
                'model_evaluation': getattr(self, 'model_evaluation', None),
                'device': str(self.device),
                'use_cupy': self.use_cupy
            }

            with open(filename, 'wb') as f:
                pickle.dump(model_data, f)

            print(f"Model saved to {filename}")
            return True
        except Exception as e:
            print(f"Error saving model: {e}")
            return False

    def load_model(self, filename):
        """Load a saved PyTorch model"""
        try:
            with open(filename, 'rb') as f:
                model_data = pickle.load(f)

            self.symbol = model_data['symbol']
            self.arima_model = model_data['arima_model']
            self.scaler = model_data['scaler']
            self.predictions = model_data['predictions']
            self.data = model_data['data_tail']

            # Recreate LSTM model
            if model_data['lstm_model_state_dict'] is not None:
                params = model_data['lstm_model_params']
                self.lstm_model = LSTMModel(
                    input_size=params['input_size'],
                    lstm_units=params['lstm_units'],
                    num_layers=params['num_layers'],
                    dropout_rate=params['dropout_rate']
                ).to(self.device)
                self.lstm_model.load_state_dict(model_data['lstm_model_state_dict'])

            print(f"Model loaded from {filename}")
            return True
        except Exception as e:
            print(f"Error loading model: {e}")
            return False

    def get_scaled_exog_features(self, date):
        """
        Fetch or compute exogenous features for the given date and scale them.
        This example assumes exogenous features include calendar-based features
        (computable for future dates) and potentially real-time fetchable features
        like stock volume or indicators if the date is in the past or current.
        """
        try:
            # Define computable calendar features
            weekday = date.weekday()  # 0-6
            month = date.month  # 1-12
            quarter = (date.month - 1) // 3 + 1

            # Placeholder for fetchable features, e.g., volume
            volume = np.nan  # Default to NaN for future dates

            current_date = datetime.now().date()  # Get current date for comparison
            if date.date() <= current_date:
                # Attempt to fetch actual data if date is not future
                try:
                    stock_data = yf.download(self.symbol, start=date, end=date + pd.Timedelta(days=1))
                    if not stock_data.empty:
                        volume = stock_data['Volume'].iloc[0]  # Get volume if available
                except:
                    pass

            # Handle NaN for volume (e.g., use mean from historical data)
            if np.isnan(volume):
                volume = self.data['Volume'].mean() if 'Volume' in self.data.columns else 0  # Fallback

            # Collect features (must match training: order and count)
            # Example: assume 3 exog features: volume, rsi, volatility (excluding residuals)
            # Simplified - in practice you'd compute actual RSI and volatility
            rsi = 50.0  # Default RSI
            volatility = self.data['Volatility'].mean() if 'Volatility' in self.data.columns else 0.02

            features = [volume, rsi, volatility]

            # Number of exog features (total scaled features - 1 for residual)
            num_exog = self.scaler.n_features_in_ - 1

            if len(features) != num_exog:
                # Pad or trim features to match expected size
                if len(features) < num_exog:
                    features.extend([0.0] * (num_exog - len(features)))
                else:
                    features = features[:num_exog]

            # Create dummy array for scaling (residual=0, then exog)
            dummy = np.zeros(self.scaler.n_features_in_)
            dummy[1:] = features

            # Scale and return only the scaled exog part
            scaled_full = self.scaler.transform(dummy.reshape(1, -1))[0]
            scaled_exog = scaled_full[1:]

            return scaled_exog

        except Exception as e:
            print(f"Error fetching/computing features for {date}: {e}")
            # Fallback: return zeros or last known scaled exog
            return np.zeros(self.scaler.n_features_in_ - 1)


In [ ]:
def train_and_save_model_pytorch(symbol, filename):
    """Complete training pipeline using PyTorch"""
    print(f"Starting PyTorch training for {symbol}...")
    print(f"Using device: {device}")
    print(f"CuPy available: {cp.cuda.is_available()}")

    forecaster = StockForecasterPyTorch(symbol)

    # Step 1: Fetch data
    if not forecaster.fetch_stock_data():
        return False

    # Step 2: Fit ARIMA
    if not forecaster.fit_arima():
        return False

    # Step 3: Train LSTM
    if not forecaster.train_lstm():
        return False

    # Step 4: Generate forecast
    predictions, news, sentiment = forecaster.forecast(30)
    if predictions is None:
        return False

    # Step 5: Evaluate model performance
    forecaster.evaluate_model_performance()

    # Step 6: Save model
    if not forecaster.save_model(filename):
        return False

    # Display results
    current_price = forecaster.data['Close'].iloc[-1]
    signals = forecaster.calculate_trading_signals(current_price)

    print("\n" + "="*50)
    print(f"PYTORCH TRAINING COMPLETED FOR {symbol}")
    print("="*50)
    print(f"Current Price: ${current_price:.2f}")
    print(f"Sentiment Score: {sentiment:.3f}")
    print(f"Recommendation: {signals['recommendation']}")
    print(f"Target Price: ${signals['target_price']:.2f}")
    print(f"Potential Profit: {signals['potential_profit_pct']:.2f}%")
    print(f"Model saved to: {filename}")

    # Plot results
    plt.figure(figsize=(15, 10))

    # Historical prices
    plt.subplot(2, 2, 1)
    plt.plot(forecaster.data.index[-100:], forecaster.data['Close'][-100:], label='Historical')
    plt.plot(predictions['date'], predictions['adjusted_forecast'], 'r--', label='Forecast')
    plt.fill_between(predictions['date'],
                     predictions['lower_bound'],
                     predictions['upper_bound'],
                     alpha=0.3)
    plt.title(f'{symbol} - Price Forecast (PyTorch)')
    plt.legend()
    plt.xticks(rotation=45)

    # ARIMA residuals
    plt.subplot(2, 2, 2)
    plt.plot(forecaster.arima_residuals)
    plt.title('ARIMA Residuals')

    # Technical indicators
    plt.subplot(2, 2, 3)
    plt.plot(forecaster.data.index[-100:], forecaster.data['RSI'][-100:])
    plt.title('RSI')
    plt.axhline(y=70, color='r', linestyle='--')
    plt.axhline(y=30, color='g', linestyle='--')

    # Volume
    plt.subplot(2, 2, 4)
    plt.bar(forecaster.data.index[-20:], forecaster.data['Volume'][-20:])
    plt.title('Volume (Last 20 days)')
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.savefig(f'{symbol}_pytorch_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

    return True


In [ ]:
# ARIMA
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

In [ ]:
if __name__ == "__main__":
    # Replace with your stock symbol and desired filename
    STOCK_SYMBOL = "AAPL"  # Change this to any stock symbol
    MODEL_FILENAME = f"{STOCK_SYMBOL}_pytorch_forecaster.pkl"

    # Train and save model
    success = train_and_save_model_pytorch(STOCK_SYMBOL, MODEL_FILENAME)

    if success:
        print("\n✅ PyTorch training completed successfully!")
        print(f"📄 Download the file '{MODEL_FILENAME}' to use in your application")
        
        # Display GPU/CuPy status
        if torch.cuda.is_available():
            print(f"🚀 GPU acceleration used: {torch.cuda.get_device_name()}")
        if cp.cuda.is_available():
            print(f"⚡ CuPy acceleration enabled")
    else:
        print("\n❌ Training failed. Please check the error messages above.")
        